In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

# Определение ключевых столбцов
id_column = 'Date'
target_column = 'Price_Increase' # Исходный целевой столбец (удалится)
close_column = 'Close'
open_column = 'Open'

# Загрузка данных (используем тот же URL из оригинальной лабы)
url = "https://raw.githubusercontent.com/hazlmar/OOP1/0daab253bfe9aa94c0cf2e0eaa33fc93d3e82ebd/IBM_Stock_1980_2025_price_increase_categorical.csv"
df = pd.read_csv(url)

print(f"Исходный размер датасета: {df.shape}")

Исходный размер датасета: (11488, 11)


In [20]:
# 3. Удаление Ненужных Столбцов и Создание Целевого Признака

# 1. Создание финального бинарного целевого признака (как в оригинале - Код 80)
# Предсказание, поднялась ли цена (Close > Open)
df['y_bin'] = (df[close_column] > df[open_column]).astype('int8')

# 2. Определение признаков (X) и целевой переменной (y)
# Столбцы, которые не должны стать признаками (X):
columns_to_exclude = [
    'Date',           # ID (удаляем)
    'Price_Increase', # Старый таргет (удаляем)
    'y_bin'           # Новый таргет (удаляем)
]

# Создаем финальный набор признаков X
X = df.drop(columns=columns_to_exclude, errors='ignore') 
y = df['y_bin']

# Обновляем списки типов столбцов для дальнейшей работы
numerical_columns = X.select_dtypes(include=np.number).columns.tolist()
categorical_columns = X.select_dtypes(include='object').columns.tolist()

print("--- 3. Удаление Ненужных Столбцов и Создание Таргета ---")
print(f"y_bin создан. Баланс классов: \n{y.value_counts(normalize=True)}")
print(f"Количество признаков (X) до дальнейшей обработки: {X.shape[1]}")
print(f"Признаки X: {X.columns.tolist()}")
print(f"Оставшиеся категориальные столбцы: {categorical_columns} (Для OHE)")
print("-" * 40)
print(len(X))

--- 3. Удаление Ненужных Столбцов и Создание Таргета ---
y_bin создан. Баланс классов: 
y_bin
0    0.509101
1    0.490899
Name: proportion, dtype: float64
Количество признаков (X) до дальнейшей обработки: 9
Признаки X: ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Dividends', 'Stock Splits', 'Adj Factor']
Оставшиеся категориальные столбцы: [] (Для OHE)
----------------------------------------
7032


In [18]:
df_temp = df.copy() # Создаем копию df после Шага 3 (где уже есть y_bin)
df_before_na = df_temp.shape[0]

# Удаляем NaNs только в столбце 'Date' (если он вообще содержал NaN)
df_temp.dropna(subset=[id_column], inplace=True)
na_removed_minimal = df_before_na - df_temp.shape[0]

# Переопределяем X и y с максимальным числом строк
X = df_temp.drop(columns=columns_to_exclude, errors='ignore')
y = df_temp['y_bin']
numerical_columns = X.select_dtypes(include=np.number).columns.tolist()

# Снова считаем NaN в X
numerical_cols_with_na = X[numerical_columns].isna().sum().sum()
nan_rows = X.isna().any(axis=1).sum()

# 5. Пропуск Выбросов
print("--- 4/5. Обновленная Обработка Пропусков и Выбросов ---")
print(f"Строк удалено (минимально, только по ID): {na_removed_minimal}")
print("❗️ Шаг 5 (Удаление Выбросов) пропущен для сохранения данных.")
print(f"Текущий размер датасета: {df_temp.shape}")
print(f"Общее количество NaN в числовых признаках (X) для ИМПУТАЦИИ: {numerical_cols_with_na}")
print(f"Количество строк с хотя бы одним NaN: {nan_rows}")
print("-" * 40)

--- 4/5. Обновленная Обработка Пропусков и Выбросов ---
Строк удалено (минимально, только по ID): 0
❗️ Шаг 5 (Удаление Выбросов) пропущен для сохранения данных.
Текущий размер датасета: (7032, 12)
Общее количество NaN в числовых признаках (X) для ИМПУТАЦИИ: 0
Количество строк с хотя бы одним NaN: 0
----------------------------------------


In [13]:
# 5. Обработка Выбросов (по Правилу 3-х Сигм)

df_before_outliers = df.shape[0]

# Функция для удаления выбросов по 3-м сигмам
def remove_outliers_3sigma(df, column):
    """Удаляет строки с выбросами в указанном столбце по правилу 3-х сигм."""
    mean = df[column].mean()
    std = df[column].std()
    lower_bound = mean - 3 * std
    upper_bound = mean + 3 * std
    
    # Сохраняем только те строки, которые находятся в пределах 3-х сигм
    df_filtered = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return df_filtered

# Применяем удаление выбросов ко всем числовым столбцам
# Важно: применяем последовательно, сохраняя результат в df
for col in numerical_columns:
    df = remove_outliers_3sigma(df, col)

outliers_removed = df_before_outliers - df.shape[0]

# Так как y и X были определены на старом df, нужно их переопределить на новом, очищенном df
X = df[X.columns] # Обновляем X, сохраняя те же столбцы
y = df['y_bin']   # Обновляем y

print("--- 5. Обработка Выбросов ---")
print(f"Общее количество удаленных строк (выбросов): {outliers_removed}")
print(f"Текущий размер датасета: {df.shape}")
print("-" * 40)

--- 5. Обработка Выбросов ---
Общее количество удаленных строк (выбросов): 3926
Текущий размер датасета: (7032, 12)
----------------------------------------


In [14]:
# 6. Кодирование Категориальных Признаков (OHE)
# Категориальные столбцы были удалены в шаге 3 (Price_Increase) или отсутствуют.

# Переопределяем X и y после удаления выбросов
# (Это гарантирует, что X и y соответствуют новому df размером 8730)
X = df[[col for col in df.columns if col not in columns_to_exclude and col != 'y_bin']] 
y = df['y_bin']

categorical_columns = X.select_dtypes(include='object').columns.tolist()

if categorical_columns:
    # Здесь был бы код OneHotEncoder
    print(f"Категориальные столбцы найдены: {categorical_columns}. Кодирование требуется.")
    pass 
else:
    print("Категориальные столбцы отсутствуют в X. Шаг кодирования пропущен.")

# Шаг 7 (Масштабирование) пропускаем, т.к. он идет после разбиения.

print("--- 6. Кодирование и Масштабирование ---")
print(f"Финальное количество признаков в X: {X.shape[1]}")
print("-" * 40)

Категориальные столбцы отсутствуют в X. Шаг кодирования пропущен.
--- 6. Кодирование и Масштабирование ---
Финальное количество признаков в X: 9
----------------------------------------


In [15]:
# Нам нужны обновленные X и y (после удаления выбросов в шаге 5)
# X = df.drop(columns=columns_to_exclude, errors='ignore')
# y = df['y_bin']

print("--- 9. Проверка Корректности Данных ---")

# 1. Проверка размера и наличия дубликатов
print(f"1. Текущий размер X: {X.shape}")
print(f"2. Дубликаты в X (после очистки): {X.duplicated().sum()}")

# 3. Проверка типов данных (должны быть только числовые)
print("\n3. Типы данных:")
print(X.dtypes.value_counts())
# Убедимся, что все столбцы числовые (float64, int64)

# 4. Проверка наличия NaN (всё ещё присутствуют)
nan_count = X.isna().sum().sum()
print(f"\n4. Общее количество NaN в X: {nan_count}")
print("   (Ожидаемо: NaN > 0, так как они будут заполнены после разбиения)")

# 5. Проверка целевого признака y (y_bin)
print("\n5. Проверка целевого признака (y_bin):")
print(f"   Тип данных y: {y.dtype}")
print(f"   Уникальные значения y: {y.unique()}")
print("   Баланс классов y:")
print(y.value_counts(normalize=True))
# Должны быть только 0 и 1

# 6. Проверка, что X и y синхронизированы (должны совпадать по индексу)
print(f"\n6. X и y синхронизированы по индексу: {X.index.equals(y.index)}")
print("-" * 40)

--- 9. Проверка Корректности Данных ---
1. Текущий размер X: (7032, 9)
2. Дубликаты в X (после очистки): 0

3. Типы данных:
float64    8
int64      1
Name: count, dtype: int64

4. Общее количество NaN в X: 0
   (Ожидаемо: NaN > 0, так как они будут заполнены после разбиения)

5. Проверка целевого признака (y_bin):
   Тип данных y: int8
   Уникальные значения y: [0 1]
   Баланс классов y:
y_bin
0    0.509101
1    0.490899
Name: proportion, dtype: float64

6. X и y синхронизированы по индексу: True
----------------------------------------


In [16]:
# 1. Разбиение на Train и Test (30%)
# X и y были синхронизированы и очищены после Шага 5
from sklearn.model_selection import train_test_split

X_train, X_test, y_train_bin, y_test_bin = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print("--- 10. Финальная Обработка: Разбиение ---")
print(f"X_train.shape: {X_train.shape}, y_train_bin.shape: {y_train_bin.shape}")
print(f"Синхронизация индексов: {X_train.index.equals(y_train_bin.index)}")
print("-" * 40)

--- 10. Финальная Обработка: Разбиение ---
X_train.shape: (4922, 9), y_train_bin.shape: (4922,)
Синхронизация индексов: True
----------------------------------------
